# **MLFA Endsem Lab Test**

Consider a classification task involving a modified MNIST data having 1000 data items distributed across 10 classes. 
Each data is an image with 20x20 dimension, flattened to 400-dimension vector. 

In [1]:
#Imports
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier

## **Question 1**

- Implement Principal Component Analysis (PCA) to perform reduction of dimensionality to 100. 

- Design a MultiLayer Perceptron (MLP) classifier that takes as input the reduced vector and predicts its class.

- Report classification accuracy by performing 5-fold cross validation. - Repeat the experiment with dimensionality reduced to 200.

In [ ]:
#Unzip MNIST data
#I am using binary format, kindly upload the below zip file in the same directory as this ipynb file
!unzip data-lab-test-13-binary.zip

In [3]:
# Load the modified MNIST dataset
X = np.load('data.npy')
y = np.load('labels.npy')

#Inspecting data dimensions
print(X.shape, y.shape)

(1000, 400) (1000,)


In [4]:
def my_pca_fit(X, n_components):
  # First we center the data
  X_centered = X - np.mean(X, axis=0)

  # Now, compute the covariance matrix
  cov_matrix = np.cov(X_centered.T)

  # Next, we compute the eigenvectors & eigenvalues of the covariance matrix
  eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

  # Sorting the eigenvectors in descending order of eigenvalues
  eigenvectors = eigenvectors[:, np.argsort(eigenvalues)[::-1]]

  # Select the first n_components
  eigenvectors = eigenvectors[:, : n_components]

  return eigenvectors

def my_pca_transform(X, eigenvectors): 
  # Again, first center the data
  X_centered = X - np.mean(X, axis=0)
  # Compute the reduced data (X_pca)
  X_pca = np.dot(X_centered, eigenvectors)
  return X_pca

Writing model in functional form so that we can use it for experiments with different parameters

In [5]:
def mlp_pca_model(X_reduced, y, n_folds, n_components):
  # Define the MLP classifier
  clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)

  # Instantiating KFold object
  kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

  # Perform cross validation
  accuracies = list()
  for train_idx, test_idx in kf.split(X):
      #Consider current fold of data
      X_train, y_train = X[train_idx], y[train_idx]
      X_test, y_test = X[test_idx], y[test_idx]
      
      #Fit PCA on X_train
      eigenvectors = my_pca_fit(X_train, n_components) 

      #Perform PCA on X_test to transform it to lower dimensionality
      X_train_pca = my_pca_transform(X_train, eigenvectors)
      
      #Perform PCA on X_test to transform it to lower dimensionality
      X_test_pca = my_pca_transform(X_test, eigenvectors)
      
      clf.fit(X_train_pca, y_train)
      curr_acc = clf.score(X_test_pca, y_test)
      accuracies.append(curr_acc)

  mean_acc = np.round(np.mean(accuracies),3)
  best_acc = np.round(np.max(accuracies),3)
  return mean_acc, best_acc

PCA - Experiment 1

In [6]:
n_folds = 5  # no. of folds for cross validation
n_components = 100  # no. of dimensions to which X will be reduced

# Train the model and evaluate it
mean_acc, best_acc = mlp_pca_model(X, y, n_folds, n_components)

# Print the average & best k-fold classification accuracy
print('Average accuracy of MLP model on data with', n_components, 'components:',mean_acc)
print('Maximum accuracy of MLP model on data with', n_components, 'components:',best_acc)

Average accuracy of MLP model on data with 100 components: 0.722
Maximum accuracy of MLP model on data with 100 components: 0.765


PCA - Experiment 2

In [7]:
n_folds = 5  # no. of folds for cross validation
n_components = 200  # no. of dimensions to which X will be reduced

# Train the model and evaluate it
mean_acc, best_acc = mlp_pca_model(X, y, n_folds, n_components)

# Print the average & best k-fold classification accuracy
print('Average accuracy of MLP model on data with', n_components, 'components:',mean_acc)
print('Maximum accuracy of MLP model on data with', n_components, 'components:',best_acc)

Average accuracy of MLP model on data with 200 components: 0.675
Maximum accuracy of MLP model on data with 200 components: 0.715


PCA - Experiment 3 (Additional)

In [8]:
n_folds = 5  # no. of folds for cross validation
n_components = 50  # no. of dimensions to which X will be reduced

# Train the model and evaluate it
mean_acc, best_acc = mlp_pca_model(X, y, n_folds, n_components)

# Print the average & best k-fold classification accuracy
print('Average accuracy of MLP model on data with', n_components, 'components:',mean_acc)
print('Maximum accuracy of MLP model on data with', n_components, 'components:',best_acc)

Average accuracy of MLP model on data with 50 components: 0.729
Maximum accuracy of MLP model on data with 50 components: 0.78


Observation: Model accuracy increasing on reducing n_components for PCA Experiments


---------------------------------------------------------------

---



## **Question 2**

- Implement Linear Discriminant Analysis (LDA) to perform reduction of dimensionality to 100. 

- Design a MultiLayer Perceptron (MLP) classifier that takes as input the reduced vector and predicts its class.

- Report classification accuracy by performing 5-fold cross validation. - Repeat the experiment with dimensionality reduced to 200.

In [9]:
# Wrting my psuedo code for LDA part to explain the core logic

# Step 1: Compute the mean vectors of each class
# Step 2: Compute within-class Scatter matrix (Sw)
# Step 3: Add regularization to Sw --> we are doing this to handle edge case of singular matrix
# Step 4: Computing between-class Scatter matrix (Sb)
# Step 5: Computing the eigenvalues & eigenvectors of Sw^-1 * Sb
# Step 6: Sorting eignvectors oin decsending order on the basis of eignvalues
# Step 7: Select the top n_components eigenvectors
# Step 8: Return W (will be later used to transform both X_train and X_test)

In [10]:
def my_lda_fit(X, y, n_components):
    # Compute the mean vectors of each class
    mean_vecs = list()
    for c in np.unique(y):
        mean_vecs.append(np.mean(X[y==c], axis=0))
    
    # Computing within-class Scatter matrix (Sw)
    dim1 = X.shape[1]
    Sw = np.zeros((dim1, dim1))
    for c, mean_vec in zip(np.unique(y), mean_vecs):
        class_scatter = np.zeros((dim1, dim1))
        for x in X[y==c]:
            x = x.reshape(-1, 1)
            mean_vec = mean_vec.reshape(-1, 1)
            class_scatter += (x - mean_vec).dot((x - mean_vec).T)
        Sw += class_scatter

    # Adding regularization to Sw --> we are doing this to handle edge case of singular matrix
    Sw += np.eye(dim1) * 1e-8
    
    # Now Computing between-class Scatter matrix (Sb)
    Sb = np.zeros((dim1, dim1))
    overall_mean = np.mean(X, axis=0)
    for c, mean_vec in zip(np.unique(y), mean_vecs):
        n = X[y==c].shape[0]
        mean_vec = mean_vec.reshape(-1, 1)
        overall_mean = overall_mean.reshape(-1, 1)
        Sb += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)
    
    # Next, we compute the eigenvalues & eigenvectors of Sw^-1 * Sb
    eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(Sw).dot(Sb))

    # Get the real part of eigenvalues and eigenvectors to remove imaginary data
    eig_vals = np.real(eig_vals)
    eig_vecs = np.real(eig_vecs)

    eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
    eig_pairs.sort(key=lambda x: x[0], reverse=True) #sorting vectors in decsending order on eignvales
    
    # Select the top n_components eigenvectors
    W = np.hstack([eig_pairs[i][1].reshape(-1,1) for i in range(n_components)])
      
    return W

   
def my_lda_transform(X, W):
    X_proj = np.dot(X, W)    # Projected data onto the LDA space
    return X_proj

In [11]:
def mlp_lda_model(X, y, n_folds, n_components):
  # Define the MLP classifier
  clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)

  # Instantiating KFold object
  kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

  # Perform cross validation
  accuracies = list()
  for train_idx, test_idx in kf.split(X):
      #Consider current fold of data
      X_train, y_train = X[train_idx], y[train_idx]
      X_test, y_test = X[test_idx], y[test_idx]
      
      #Fit LDA on X_train
      W = my_lda_fit(X_train, y_train, n_components) 

      #Perform LDA on X_test to transform it to lower dimensionality
      X_train_lda = my_lda_transform(X_train, W)
      
      #Perform LDA on X_test to transform it to lower dimensionality
      X_test_lda = my_lda_transform(X_test, W)
      
      clf.fit(X_train_lda, y_train)
      curr_acc = clf.score(X_test_lda, y_test)
      accuracies.append(curr_acc)

  mean_acc = np.round(np.mean(accuracies),3)
  best_acc = np.round(np.max(accuracies),3)
  return mean_acc, best_acc

LDA - Experiment 1

In [16]:
n_folds = 5  # no. of folds for cross validation
n_components = 100  # no. of dimensions to which X will be reduced

# Train the model and evaluate it
mean_acc, best_acc = mlp_lda_model(X, y, n_folds, n_components)

# Print the average & best k-fold classification accuracy
print('Average accuracy of MLP model on data with', n_components, 'components:',mean_acc)
print('Maximum accuracy of MLP model on data with', n_components, 'components:',best_acc)

Average accuracy of MLP model on data with 100 components: 0.623
Maximum accuracy of MLP model on data with 100 components: 0.67


LDA - Experiment 2

In [13]:
n_folds = 5  # no. of folds for cross validation
n_components = 200  # no. of dimensions to which X will be reduced

# Train the model and evaluate it
mean_acc, best_acc = mlp_lda_model(X, y, n_folds, n_components)

# Print the average & best k-fold classification accuracy
print('Average accuracy of MLP model on data with', n_components, 'components:',mean_acc)
print('Maximum accuracy of MLP model on data with', n_components, 'components:',best_acc)

Average accuracy of MLP model on data with 200 components: 0.613
Maximum accuracy of MLP model on data with 200 components: 0.68
